In [ ]:
%matplotlib inline
import math
import sympy 
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
from cv2 import *
import cv2
import copy
import time
from numba import jit, cuda
from Crypto.Signature import PKCS1_v1_5
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA3_256
from Crypto.Hash import SHA256
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FileUpload, Button
from ipywidgets import Checkbox, Output
from ipywidgets import Dropdown, FileUpload, Text, HBox, VBox


In [ ]:




@jit(target_backend='cuda')
def data_manipulation(
    frame1=np.array([]),
    frame2=np.array([]),
    frame3=np.array([]),
    frame4=np.array([]),
    frame5=np.array([]),
    frame6=np.array([]),
    frame7=np.array([]),
    frame8=np.array([]),
    frame9=np.array([]),
    frame10=np.array([]),
    ):
    height=21600 
    width=384
    
    frame1 = frame1.reshape((height, width*3)) 
    frame2 = frame2.reshape((height, width*3)) 
    frame3 = frame3.reshape((height, width*3)) 
    frame4 = frame4.reshape((height, width*3)) 
    frame5 = frame5.reshape((height, width*3)) 
    frame6 = frame6.reshape((height, width*3)) 
    frame7 = frame7.reshape((height, width*3)) 
    frame8 = frame8.reshape((height, width*3)) 
    frame9 = frame9.reshape((height, width*3)) 
    frame10 = frame10.reshape((height, width*3)) 
    

    frame1 = np.transpose(frame1)
    frame2 = np.transpose(frame2)
    frame3 = np.transpose(frame3)
    frame4 = np.transpose(frame4)
    frame5 = np.transpose(frame5)
    frame6 = np.transpose(frame6)
    frame7 = np.transpose(frame7)
    frame8 = np.transpose(frame8)
    frame9 = np.transpose(frame9)
    frame10 = np.transpose(frame10)
    
       
    frame1 = frame1.ravel()
    frame2 = frame2.ravel()
    frame3 = frame3.ravel()
    frame4 = frame4.ravel()
    frame5 = frame5.ravel()
    frame6 = frame6.ravel()
    frame7 = frame7.ravel()
    frame8 = frame8.ravel()
    frame9 = frame9.ravel()
    frame10 = frame10.ravel()
    
    frame_interweaved = np.empty(
        (frame1.size + frame2.size + frame3.size + frame4.size + frame5.size+
         frame6.size + frame7.size + frame8.size + frame9.size + frame10.size
        ), dtype=frame1.dtype)
    
    frame_interweaved[0::10] = frame1
    frame_interweaved[1::10] = frame2
    frame_interweaved[2::10] = frame3
    frame_interweaved[3::10] = frame4
    frame_interweaved[4::10] = frame5
    frame_interweaved[5::10] = frame6
    frame_interweaved[6::10] = frame7
    frame_interweaved[7::10] = frame8
    frame_interweaved[8::10] = frame9
    frame_interweaved[9::10] = frame10
               
    return  frame_interweaved;



@jit(target_backend='cuda')
def remove_and_cut_pixels_GPU(bit_array=[]):
    pixel_array_removed_pixel = np.empty(len(bit_array), dtype=np.uint8)
    i = 0 
    for number in bit_array:
        if number >= 2 and number <= 253: 
            pixel_array_removed_pixel[i] = (number)%2
            i +=1    
    return pixel_array_removed_pixel[:i];


@jit(target_backend='cuda',fastmath=True)
def equalize_bit_count(bit_array=np.array([])):
    pixel_array_removed_pixel = np.empty(len(bit_array), dtype=np.uint8)
    zeros = np.count_nonzero(bit_array == 0)
    ones = np.count_nonzero(bit_array == 1)
    difference = ones - zeros
    if difference > 0:
        factor = difference/ones
        one_count = 0
        i = 0 
        for bit in bit_array:
            if bit == 1:
                if one_count > 1:
                    one_count -=1
                else:
                    one_count+=factor
                    pixel_array_removed_pixel[i] = bit
                    i +=1 
            else:
                pixel_array_removed_pixel[i] = bit
                i +=1
            
    elif difference < 0:  
        factor = (-1*difference)/zeros
        zero_count = 0
        i = 0 
        for bit in bit_array:
            if bit == 0:
                if zero_count > 1:
                    zero_count -=1
                else:
                    zero_count+=factor
                    pixel_array_removed_pixel[i] = bit
                    i +=1 
            else:
                pixel_array_removed_pixel[i] = bit
                i +=1
    return pixel_array_removed_pixel[:i];


@jit(target_backend='cuda')
def merge_bits_GPU(bit_array=np.array([])):
    bit_array_merged = np.empty(len(bit_array), dtype=np.uint64)
    i = 0 
    for bit_pack in bit_array:
        random_value = 0 
        for bit in bit_pack:
            random_value = (random_value << 1) | int(bit)
        bit_array_merged[i] = int(random_value)
        i +=1
    
    return bit_array_merged


video1 = cv2.VideoCapture("Pictures/rng_video_1.mp4")
video2 = cv2.VideoCapture("Pictures/rng_video_2.mp4")
video3 = cv2.VideoCapture("Pictures/rng_video_3.mp4")
video4 = cv2.VideoCapture("Pictures/rng_video_4.mp4")
video5 = cv2.VideoCapture("Pictures/rng_video_5.mp4")
video6 = cv2.VideoCapture("Pictures/rng_video_6.mp4")
video7 = cv2.VideoCapture("Pictures/rng_video_7.mp4")
video8 = cv2.VideoCapture("Pictures/rng_video_8.mp4")
video9 = cv2.VideoCapture("Pictures/rng_video_9.mp4")
video10 = cv2.VideoCapture("Pictures/rng_video_10.mp4")
ret1, frame1 = video1.read()    
ret2, frame2 = video2.read()  
ret3, frame3 = video3.read()  
ret4, frame4 = video4.read()  
ret5, frame5 = video5.read()  
ret6, frame6 = video6.read()  
ret7, frame7 = video7.read()  
ret8, frame8 = video8.read()  
ret9, frame9 = video9.read()  
ret10, frame10 = video10.read() 


random_numbers2 = np.empty(0, int)
how_many_numbers = 100000000
current_numbers_count = 0
while current_numbers_count < how_many_numbers:

    start = time.time()
    ret1, frame1 = video1.read()    
    ret2, frame2 = video2.read()  
    ret3, frame3 = video3.read()  
    ret4, frame4 = video4.read()  
    ret5, frame5 = video5.read()  
    ret6, frame6 = video6.read()  
    ret7, frame7 = video7.read()  
    ret8, frame8 = video8.read()  
    ret9, frame9 = video9.read()  
    ret10, frame10 = video10.read() 

    
    #0
    frame_all = np.concatenate((frame1.reshape(2160,3840,3),frame2,frame3,frame4,frame5,frame6,frame7,frame8,frame9,frame10))    
    pixel_array = frame_all.ravel()

    
    #1
    bit_array = data_manipulation(frame1, frame2, frame3, frame4, frame5, frame6, frame7, frame8, frame9, frame10)
    
    
    #2
    bit_array = remove_and_cut_pixels_GPU(bit_array)


    #4  
    bit_array = equalize_bit_count(bit_array)


    #5
    random_bit_length =  8
    random_in_image, extra_bits = divmod(len(bit_array),random_bit_length)
    if extra_bits > 0:
        bit_array = bit_array[:-extra_bits]

    bit_array=bit_array.reshape(random_in_image, random_bit_length)

    
    random_numbers = merge_bits_GPU(bit_array)
    
    random_numbers2 = np.concatenate((random_numbers2, random_numbers), axis=0)
    current_numbers_count= len(random_numbers2)
    

random_numbers2= random_numbers2[:how_many_numbers]


In [ ]:
global start_point
global print_more
print_more = False
start_point = 0
def random_with_range(start,stop):
    global start_point
    n = stop.bit_length()
    not_found = True
    while not_found:
        random_value = 0 

        current_bit_number = 0;
        while current_bit_number < n:
            random_value = (random_value << 8) | int(random_numbers2[start_point])
            current_bit_number+=8
            start_point+=1



        if(random_value >= start and random_value < stop):
            not_found = False
    return random_value
def generate_random_prime(n):
    global start_point
    not_found = True
    while not_found:
        random_value = 0 

        current_bit_number = 0;
        while current_bit_number < n:
            random_value = (random_value << 8) | int(random_numbers2[start_point])
            current_bit_number+=8
            start_point+=1
 

        # Set the most significant bit and the least significant bit
        random_value |= (1 << (n-1)) | 1
        
        #Miller-Rabin primality test        
        if is_prime(random_value):
            not_found = False
            return random_value

def is_prime(num, k=40):
    # small prime numbers
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
    if num in small_primes:
        return True
    if num <= 1 or any(num % prime == 0 for prime in small_primes):
        return False
    
    #Miller-Rabin primality test
    r, s = 0, num - 1
    while s % 2 == 0:
        r += 1
        s //= 2

    for _ in range(k):
        a = random_with_range(2, num - 1)
        x = pow(a, s, num)

        if x == 1 or x == num - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, num)
            if x == num - 1:
                break
        else:
            return False
    return True




In [ ]:
global keys
keys = 0

def generate_keys(bit_length=2048):
    global print_more
    #1
    p = generate_random_prime(bit_length)
    q = generate_random_prime(bit_length)
    #2
    n = p*q


    #3
    lambda_n = math.lcm(p-1,q-1)

    #4
    e= 65537


    d = pow(e, -1, lambda_n)
    

    global keys
    keys = RSA.construct((n,e,d,p,q))
    
    with open("whole_key.pem", "wb") as file:
        file.write(keys.export_key("PEM"))
    
    public_key = keys.public_key()
    with open("public_key.pem", "wb") as file:
        file.write(public_key.export_key("PEM"))
          
            
    if print_more == True:
        print("n =", n)
        print("e =", e)
        print("d =", d)
        print(f'Public key: {e, n}')
        print(f'Private key: {d, n}')

    print("Wygenerowano klucze") 


def sign_message(file_path):
    with open("whole_key.pem", "rb") as file:
        keys = RSA.import_key(file.read()) 
    
    
    
    
    hash_obj = SHA256.new() 
    

    with open(file_path, 'rb') as file:
        chunk = file.read(4096)  
        while len(chunk) > 0:
            hash_obj.update(chunk)
            chunk = file.read(4096)
    

    
    

    signer = PKCS1_v1_5.new(keys)
    signature = signer.sign(hash_obj)


    global print_more

    with open('signature.bin', 'wb') as f:    
        f.write(signature)
        print("Zapisano sygnature")



def verify_signature(file_path):
    public_key = 0
    with open("public_key.pem", "rb") as file:
        public_key = RSA.import_key(file.read()) 
    

    f3 = open("signature.bin", "rb")
    signature = (f3.read()) 
    
    
    hash_obj = SHA256.new()  
    

    with open(file_path, 'rb') as file:
        chunk = file.read(4096)  
        while len(chunk) > 0:
            hash_obj.update(chunk)
            chunk = file.read(4096)
    

    verifier = PKCS1_v1_5.new(public_key)
    
    if verifier.verify(hash_obj, signature):
        print("Signature is valid.")
    else:
        print("Signature is invalid.")
        






In [ ]:
global key_bit_length
key_bit_length = 128
file_name_1 = ""
file_name_2 = ""


file_input_1 = Text(description='nazwa pliku 1')
file_input_2 = Text(description='nazwa pliku 2')
input_box = HBox([file_input_1, file_input_2])
display(VBox([file_input_1, file_input_2]))



options = [128, 256, 512, 1024, 2048]
dropdown = Dropdown(options=options, description='Długość kluczy')
def handle_dropdown_change(change):
    global key_bit_length
    key_bit_length = change['new']
dropdown.observe(handle_dropdown_change, names='value')
display(dropdown)



checkbox = Checkbox(description='Wypisanie dodatkowych danych')
def handle_checkbox_change(change):
    global print_more
    if change['new']:
        print_more = True
    else:
        print_more = False
checkbox.observe(handle_checkbox_change, names='value')
display(checkbox)




button_1 = Button(description="Wygeneruj klucze")
def on_button_click_1(button_1):
    output.clear_output()
    generate_keys(key_bit_length)    
button_1.on_click(on_button_click_1)
display(button_1, output)




button_2 = Button(description="Zaszyfruj skrót")
def on_button_click_2(button_2):
    path = 'Pictures/'+str(file_input_1.value)
    sign_message(path)
button_2.on_click(on_button_click_2)
display(button_2, output)        







button_3 = Button(description="Odszyfruj skrót")
def on_button_click_3(button_3):
    path2 = 'Pictures/'+str(file_input_2.value)
    verify_signature(path2)
button_3.on_click(on_button_click_3)
display(button_3, output)
